In [33]:
import os
log_dir = '/vol/vssp/mammo2/will/logs/new'
fCount = 0
while os.path.exists(os.path.join(log_dir, 'training', str(fCount))):
    fCount+=1
    
print(os.path.exists(os.path.join(log_dir, 'training')))
print(log_dir + 'hello')

True
/vol/vssp/mammo2/will/logs/newhello


In [1]:
# fold
import cv2
import glob
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import optimizers
# For some reason I have to tell it to use TensorFlows dimension ordering
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.callbacks import TensorBoard
from time import time
import os
import tensorflow as tf
from keras import applications

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='/vol/vssp/mammo2/will/logs/new', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        fCount=0
        while os.path.exists(os.path.join(log_dir, 'training' + '_' + str(fCount))):
            fCount+=1
        training_log_dir = os.path.join(log_dir, 'training' + '_' + str(fCount))
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation' + '_' + str(fCount))

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()


def get_images(path, dataSpecs):
    fileList = glob.glob(path) #'BengaliBMPConvert/*.bmp'   
    num = len(fileList)
    dataSpecs['classLength'].append(len(fileList))
    x = np.array([(cv2.imread(fname)) for fname in fileList])
    return x

def get_labels_one_hot(num_classes, class_id, num_samples):
    x = np.zeros((num_samples, num_classes))
    x[np.arange(num_samples),class_id] = 1
    return x

def fourCNN():
    model = Sequential()
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(385, 385, 1), data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(32, (3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(32, (3,3), activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    # Layer 5     
    #model.add(Conv2D(32, (3,3), activation='relu'))   
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 6
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    return model
    
def fiveCNN():
    model = Sequential()
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(385, 385, 1), data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(32, (3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(32, (3,3), activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    # Layer 5     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 6
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))  
    return model
    
    
def main():
    import keras
    print('keras version: ', keras.__version__)
    print('TensorFlow version: ', tf.__version__)
    print('\nLoad images...')
    
    # Get images and labels
    data = {'img': 0, 'label': 0}
    dataSpecs = {'classLength': []}
    dataSpecs['classLength'] = []
    data['img'] = np.concatenate((
            #get_images('/vol/vssp/mammo2/will/data/simulated/calcs/small_sample/0/*', dataSpecs), # Class 0
            #get_images('/vol/vssp/mammo2/will/data/simulated/calcs/small_sample/1/*', dataSpecs) # Class 1
            get_images('/user/HS204/wm0015/student/allCalcs/0/*', dataSpecs), # Class 0
            get_images('/user/HS204/wm0015/student/allCalcs/1/*', dataSpecs) # Class 1
    ))      

    # Normalise
    data['img'] = data['img']/255    
    # Print image    
    img_calc = data['img']   
    plt.imshow(img_calc[0], cmap='gray')
    plt.show()
    
    # Create one hot labels
    labels_bg = get_labels_one_hot(2, 0, dataSpecs['classLength'][0])  
    labels_calc = get_labels_one_hot(2, 1, dataSpecs['classLength'][1])
    data['label'] = np.concatenate((
            get_labels_one_hot(2, 0, dataSpecs['classLength'][0]), # Class 0 
            get_labels_one_hot(2, 1, dataSpecs['classLength'][1]) # Class 1
    ))
    # Drop from 3 colour channels to 1 (greyscale)
    if 1==1:
        data['img'] = data['img'][:,:,:,0]
        data['img'] = np.reshape(data['img'], (data['img'].shape[0],data['img'].shape[1],data['img'].shape[2],1))
        print('new data shape = ', data['img'].shape)
    
    valStats = []
    folds = 10
    for crossVal in range(folds):

        # Shuffle data
        seed = 33
    #     np.random.seed(seed) # Has to be set before each use of random
        shuffleMask = np.random.permutation(data['img'].shape[0])    
        data['img'] = data['img'][shuffleMask, :, :, :]
        data['label'] = data['label'][shuffleMask, :]

        # Split traing and validation data        
        splitRatio = 0.9
        splitPoint = math.floor(data['img'].shape[0]*splitRatio)
        train_data = {'img': data['img'][0:splitPoint], 'label': data['label'][0:splitPoint]}
        val_data = {'img': data['img'][splitPoint:], 'label': data['label'][splitPoint:]}
#         print('train_data[img].shape = ', train_data['img'].shape)
#         print('train_data[label].shape = ', train_data['label'].shape)
#         print('val_data[img].shape = ', val_data['img'].shape)
#         print('valdiation_data[label].shape = ', val_data['label'].shape)
#         print('dataSpecs = ', dataSpecs['classLength'])


        sgd = optimizers.SGD(lr=5, decay=1e-6, momentum=0.9, nesterov=True)
        adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #0.001
        
        model = fiveCNN()
        model.compile(loss='binary_crossentropy',
                    optimizer=adam,
                     metrics=['accuracy'])
        tensorboard = TensorBoard(log_dir='/vol/vssp/mammo2/will/logs/new'.format(time()), write_images=True)
        if folds == 1:
            head.summary()  

        # Train
        model.fit(train_data['img'], train_data['label'], 
                batch_size=100, epochs=150, verbose=1,
                callbacks=[TrainValTensorBoard(write_graph=False)],
                validation_data=(val_data['img'], val_data['label']))

        # Evaluate
        score = model.evaluate(val_data['img'], val_data['label'], verbose=0)
        valStats.append(score)
        print('The score is......\n', score)


    valStats = np.asarray(valStats)
    print('Validations: \n', valStats[:, 1])
    print('Average loss: ', sum(valStats[:, 0])/folds)
    print('Average validation: ', sum(valStats[:, 1])/folds)
    
if __name__ == "__main__":
    main()

Using TensorFlow backend.


keras version:  2.2.0
TensorFlow version:  1.7.1

Load images...


<Figure size 640x480 with 1 Axes>

new data shape =  (1620, 385, 385, 1)
Train on 1458 samples, validate on 162 samples
Epoch 1/150
1458/1458 [==============================] - 8s 6ms/step - loss: 0.6886 - acc: 0.5508 - val_loss: 0.6764 - val_acc: 0.5802
Epoch 2/150
1458/1458 [==============================] - 5s 4ms/step - loss: 0.6829 - acc: 0.5542 - val_loss: 0.6838 - val_acc: 0.6914
Epoch 3/150
1458/1458 [==============================] - 6s 4ms/step - loss: 0.6747 - acc: 0.6022 - val_loss: 0.6596 - val_acc: 0.7469
Epoch 4/150
1458/1458 [==============================] - 5s 4ms/step - loss: 0.6256 - acc: 0.6646 - val_loss: 0.5631 - val_acc: 0.7222
Epoch 5/150
1458/1458 [==============================] - 6s 4ms/step - loss: 0.5462 - acc: 0.7188 - val_loss: 0.6098 - val_acc: 0.6111
Epoch 6/150
1458/1458 [==============================] - 6s 4ms/step - loss: 0.4856 - acc: 0.7634 - val_loss: 0.4857 - val_acc: 0.7469
Epoch 7/150
1458/1458 [==============================] - 6s 4ms/step - loss: 0.3606 - acc: 0.8409 - val_l

1458/1458 [==============================] - 6s 4ms/step - loss: 3.5034e-05 - acc: 1.0000 - val_loss: 0.0837 - val_acc: 0.9815
Epoch 60/150
1458/1458 [==============================] - 6s 4ms/step - loss: 2.4399e-05 - acc: 1.0000 - val_loss: 0.0830 - val_acc: 0.9815
Epoch 61/150
1458/1458 [==============================] - 5s 4ms/step - loss: 1.9311e-05 - acc: 1.0000 - val_loss: 0.0813 - val_acc: 0.9815
Epoch 62/150
1458/1458 [==============================] - 6s 4ms/step - loss: 4.4254e-05 - acc: 1.0000 - val_loss: 0.0770 - val_acc: 0.9815
Epoch 63/150
1458/1458 [==============================] - 5s 4ms/step - loss: 5.4105e-05 - acc: 1.0000 - val_loss: 0.0846 - val_acc: 0.9815
Epoch 64/150
1458/1458 [==============================] - 5s 4ms/step - loss: 2.0509e-05 - acc: 1.0000 - val_loss: 0.0797 - val_acc: 0.9815
Epoch 65/150
1458/1458 [==============================] - 5s 4ms/step - loss: 1.5720e-05 - acc: 1.0000 - val_loss: 0.0804 - val_acc: 0.9815
Epoch 66/150
1458/1458 [=========

...

1458/1458 [==============================] - 5s 4ms/step - loss: 1.6043e-05 - acc: 1.0000 - val_loss: 0.0502 - val_acc: 0.9815
Epoch 127/150
1458/1458 [==============================] - 5s 4ms/step - loss: 2.2641e-05 - acc: 1.0000 - val_loss: 0.0590 - val_acc: 0.9815
Epoch 128/150
1458/1458 [==============================] - 5s 3ms/step - loss: 1.4563e-05 - acc: 1.0000 - val_loss: 0.0580 - val_acc: 0.9815
Epoch 129/150
1458/1458 [==============================] - 5s 4ms/step - loss: 1.7616e-05 - acc: 1.0000 - val_loss: 0.0592 - val_acc: 0.9815
Epoch 130/150
1458/1458 [==============================] - 5s 4ms/step - loss: 8.6938e-06 - acc: 1.0000 - val_loss: 0.0560 - val_acc: 0.9815
Epoch 131/150
1458/1458 [==============================] - 5s 4ms/step - loss: 9.8557e-06 - acc: 1.0000 - val_loss: 0.0543 - val_acc: 0.9815
Epoch 132/150
1458/1458 [==============================] - 5s 4ms/step - loss: 1.0086e-05 - acc: 1.0000 - val_loss: 0.0553 - val_acc: 0.9815
Epoch 133/150
1458/1458 [==

1458/1458 [==============================] - 6s 4ms/step - loss: 1.3235e-04 - acc: 1.0000 - val_loss: 0.1021 - val_acc: 0.9691
Epoch 35/150
1458/1458 [==============================] - 6s 4ms/step - loss: 1.4512e-04 - acc: 1.0000 - val_loss: 0.0759 - val_acc: 0.9691
Epoch 36/150
1458/1458 [==============================] - 6s 4ms/step - loss: 1.4168e-04 - acc: 1.0000 - val_loss: 0.0917 - val_acc: 0.9691
Epoch 37/150
1458/1458 [==============================] - 6s 4ms/step - loss: 7.7332e-05 - acc: 1.0000 - val_loss: 0.0876 - val_acc: 0.9691
Epoch 38/150
1458/1458 [==============================] - 6s 4ms/step - loss: 4.8612e-05 - acc: 1.0000 - val_loss: 0.0865 - val_acc: 0.9691
Epoch 39/150
1458/1458 [==============================] - 6s 4ms/step - loss: 6.5763e-04 - acc: 0.9993 - val_loss: 0.0333 - val_acc: 0.9877
Epoch 40/150
1458/1458 [==============================] - 6s 4ms/step - loss: 0.0074 - acc: 0.9966 - val_loss: 0.0160 - val_acc: 0.9938
Epoch 41/150
1458/1458 [=============

Epoch 93/150
1458/1458 [==============================] - 6s 4ms/step - loss: 5.4950e-05 - acc: 1.0000 - val_loss: 0.0748 - val_acc: 0.9877
Epoch 94/150
1458/1458 [==============================] - 6s 4ms/step - loss: 2.8039e-05 - acc: 1.0000 - val_loss: 0.0788 - val_acc: 0.9877
Epoch 95/150
1458/1458 [==============================] - 6s 4ms/step - loss: 2.8365e-04 - acc: 1.0000 - val_loss: 0.0955 - val_acc: 0.9877
Epoch 96/150
1458/1458 [==============================] - 6s 4ms/step - loss: 1.2782e-04 - acc: 1.0000 - val_loss: 0.0954 - val_acc: 0.9877
Epoch 97/150
1458/1458 [==============================] - 6s 4ms/step - loss: 1.7726e-05 - acc: 1.0000 - val_loss: 0.0824 - val_acc: 0.9877
Epoch 98/150
1458/1458 [==============================] - 6s 4ms/step - loss: 3.5429e-05 - acc: 1.0000 - val_loss: 0.0778 - val_acc: 0.9877
Epoch 99/150
1458/1458 [==============================] - 6s 4ms/step - loss: 0.0012 - acc: 0.9993 - val_loss: 0.2161 - val_acc: 0.9691
Epoch 100/150
1458/1458 

Epoch 1/150
1458/1458 [==============================] - 6s 4ms/step - loss: 0.6937 - acc: 0.5480 - val_loss: 0.6791 - val_acc: 0.5741
Epoch 2/150
1458/1458 [==============================] - 6s 4ms/step - loss: 0.6770 - acc: 0.5542 - val_loss: 0.6554 - val_acc: 0.5741
Epoch 3/150
1458/1458 [==============================] - 5s 4ms/step - loss: 0.6542 - acc: 0.5610 - val_loss: 0.6591 - val_acc: 0.8395
Epoch 4/150
1458/1458 [==============================] - 6s 4ms/step - loss: 0.6299 - acc: 0.6708 - val_loss: 0.5630 - val_acc: 0.5741
Epoch 5/150
1458/1458 [==============================] - 6s 4ms/step - loss: 0.5326 - acc: 0.7476 - val_loss: 0.4357 - val_acc: 0.8827
Epoch 6/150
1458/1458 [==============================] - 6s 4ms/step - loss: 0.3889 - acc: 0.8663 - val_loss: 0.3034 - val_acc: 0.8704
Epoch 7/150
1458/1458 [==============================] - 6s 4ms/step - loss: 0.3088 - acc: 0.8711 - val_loss: 0.2397 - val_acc: 0.9136
Epoch 8/150
1458/1458 [==============================] 

KeyboardInterrupt: 

In [2]:
print('hello world')

hello world


In [12]:
x = [[0.6940992025681484, 0.3950617283950617], [0.6884314866713536, 0.5185185185185185], [0.6855149121932042, 0.5679012345679012]]
print(x[0])
print(sum(x[:][1])/2)

[0.6940992025681484, 0.3950617283950617]
0.603475002594936


In [35]:
x = [[6.5308382717179665, 0.5925925925925926], [0.16470135086112553, 0.9444444444444444], [0.15422181794304907, 0.9444444444444444], [0.19875195217721256, 0.9382716049382716]] 
x = np.asarray(x)
print(x, '\n')
print(x[0][0])
print(x[:, 0])
print(x.shape)

[[6.53083827 0.59259259]
 [0.16470135 0.94444444]
 [0.15422182 0.94444444]
 [0.19875195 0.9382716 ]] 

6.5308382717179665
[6.53083827 0.16470135 0.15422182 0.19875195]
(4, 2)
